In [35]:
import pandas as pd
from nltk.sentiment.vader import SentimentIntensityAnalyzer
import nltk
import re

In [36]:
# VADER 사전 다운로드 (최초 한 번만 실행)
nltk.download('vader_lexicon')

[nltk_data] Downloading package vader_lexicon to
[nltk_data]     C:\Users\82102\AppData\Roaming\nltk_data...
[nltk_data]   Package vader_lexicon is already up-to-date!


True

In [37]:
# CSV 파일 불러오기
df = pd.read_csv("user_@ma_tweets.csv")

In [38]:
# URL 제거 함수
def remove_urls(text):
    return re.sub(r'https?://\S+', '', text).strip()

In [39]:
# URL 제거 적용
df['full_text'] = df['full_text'].apply(remove_urls)

In [40]:
# 빈 문자열 → NaN
df['full_text'].replace('', pd.NA, inplace=True)

C:\Users\82102\AppData\Local\Temp\ipykernel_13548\1257741985.py:2: FutureWarning: A value is trying to be set on a copy of a DataFrame or Series through chained assignment using an inplace method.
The behavior will change in pandas 3.0. This inplace method will never work because the intermediate object on which we are setting values always behaves as a copy.

For example, when doing 'df[col].method(value, inplace=True)', try using 'df.method({col: value}, inplace=True)' or df[col] = df[col].method(value) instead, to perform the operation inplace on the original object.


  df['full_text'].replace('', pd.NA, inplace=True)


In [41]:
# NaN 제거 (필요 시)
df.dropna(subset=['full_text'], inplace=True)

In [42]:
# 결과 확인
print(df.head())

                       created_at  \
0  Fri May 23 09:50:40 +0000 2025   
1  Thu May 22 02:57:22 +0000 2025   
2  Mon May 12 12:25:39 +0000 2025   
3  Sat May 10 13:06:26 +0000 2025   
4  Sat May 10 13:06:22 +0000 2025   

                                           full_text  
0     100 days until we’re back at home 🐊  #GoGators  
1  The pro-Maduro Biden oil  license in #Venezuel...  
2  .@POTUS is fighting to secure the release of e...  
3  Pakistan and India  have agreed to a ceasefire...  
4  India and Pakistan have today worked out an un...  


In [43]:
# SentimentIntensityAnalyzer 초기화
sia = SentimentIntensityAnalyzer()

In [44]:
# 감성 분석 함수
def analyze_sentiment(text):
    scores = sia.polarity_scores(text)
    compound = scores['compound']
    if compound >= 0.05:
        sentiment = 'positive'
    elif compound <= -0.05:
        sentiment = 'negative'
    else:
        sentiment = 'neutral'
    return pd.Series([sentiment, compound])

In [45]:
# 감성 분석 결과 적용 및 컬럼 추가
df[['sentiment', 'compound']] = df['full_text'].apply(analyze_sentiment)

In [46]:
# 필요한 컬럼만 추출
df = df[['created_at', 'full_text', 'sentiment', 'compound']]

In [47]:
# 결과 저장
df.to_csv("tweets_with_sentiment_@marcorubio.csv", index=False)